In [ ]:
%pip install -r ../requirements.txt --quiet

In [1]:
# 1. Imports and Setup
import time
import json
import logging
from typing import List, Dict, Any

import pandas as pd
import openai
import yaml
from tqdm.auto import tqdm

In [5]:
# 2. Load API Key from secrets.yaml
# secrets.yaml should contain:
# OPENAI_API_KEY: "your_api_key_here"
with open("../configs/secrets.yaml", "r") as f:
    secrets = yaml.safe_load(f)
openai.api_key = secrets.get("OPENAI_API_KEY")
if not openai.api_key:
    raise ValueError("OPENAI_API_KEY missing in secrets.yaml")

In [6]:
# 3. Configuration Constants
MODEL = "gpt-4o-mini"  # more powerful model
BATCH_SIZE = 50          # ~20 calls for 1,000 rows
PRICE_PER_1K_INPUT = 0.0015
PRICE_PER_1K_OUTPUT = 0.002
MAX_RETRIES = 5
BACKOFF_FACTOR = 2

# Function spec for function-calling API
FUNCTIONS: List[Dict[str, Any]] = [
    {
        "name": "score_abstracts",
        "description": "Scores a list of abstracts based on defined criteria.",
        "parameters": {
            "type": "object",
            "properties": {
                "results": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "BoothNumber": {"type": "string"},
                            "Originality": {"type": "integer"},
                            "ScientificRigor": {"type": "integer"},
                            "Clarity": {"type": "integer"},
                            "Relevance": {"type": "integer"},
                            "Feasibility": {"type": "integer"},
                        },
                        "required": [
                            "BoothNumber",
                            "Originality",
                            "ScientificRigor",
                            "Clarity",
                            "Relevance",
                            "Feasibility",
                        ],
                    },
                }
            },
            "required": ["results"],
        },
    }
]

In [15]:
# 4. Helper Functions

def chunk_list(lst: List[Any], size: int) -> List[List[Any]]:
    """Split a list into chunks of given size."""
    return [lst[i : i + size] for i in range(0, len(lst), size)]


def call_api_with_retries(
    messages: List[Dict[str, str]],
    functions: List[Dict[str, Any]],
    model: str,
) -> Any:
    """Call OpenAI Chat Completion with retries on failure."""
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            return openai.chat.completions.create(
                model=model,
                messages=messages,
                functions=functions,
                function_call={"name": functions[0]["name"]},
                temperature=0.0,
            )
        except Exception as e:
            wait = BACKOFF_FACTOR ** attempt
            logging.warning(
                f"API error on attempt {attempt}/{MAX_RETRIES}: {e}. Retrying in {wait}s..."
            )
            time.sleep(wait)
    raise RuntimeError("OpenAI API requests failed after retries.")


def evaluate_batch(batch_df: pd.DataFrame) -> List[Dict[str, Any]]:
    """Evaluate a batch of abstracts via function-calling."""
    payload = [
        {"BoothNumber": str(row.BoothNumber), "AbstractText": row.AbstractText}
        for _, row in batch_df.iterrows()
    ]
    system_msg = (
        "You are an expert evaluator. For each abstract, score:"
        " Originality/Innovation, Scientific Rigor/Methodology,"
        " Clarity/Communication, Relevance/Impact, Feasibility/Execution."
        " Return 1-10 integers via the provided function."
    )
    user_msg = f"Evaluate these abstracts as JSON.\n{json.dumps(payload, indent=2)}"
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_msg},
    ]

    resp = call_api_with_retries(messages, FUNCTIONS, MODEL)
    args = json.loads(resp.choices[0].message.function_call.arguments)
    return args.get("results", []), resp.usage

In [16]:
# 5. Load Data
input_path = "2025 ISEF Project Abstracts.csv"
output_path = "Sorted by Score - 2025 ISEF Project Abstracts.csv"

df = pd.read_csv(input_path)
df["BoothNumber"] = df["BoothNumber"].astype(str)

In [17]:
df.head()

,BoothNumber,ParentCategory,ProjectTitle,AbstractText
0,ANIM001,Animal Sciences,Investigating the Synergistic Effects of High-...,The project targeted two specific nutrients ty...
1,ANIM002,Animal Sciences,Evaluating the Efficacy of Novel Carbon Dioxid...,"Honeybees are indispensable pollinators, contr..."
2,ANIM003,Animal Sciences,Circadian Evolution in Action: How Latitude Sh...,The circadian rhythm is a 24-hour biological c...
3,ANIM004T,Animal Sciences,Tube-Worm Hunters: Ecological Aspects of Ficop...,Non-native species pose a global threat to aqu...
4,ANIM005,Animal Sciences,PawPath: An IMU-Based Gait Detection and Disea...,"PawPath is a non-invasive, risk-free gait moni..."


In [18]:
# 6. Batch Evaluation Loop
results = []
total_cost = 0.0
start_time = time.time()

for batch_idxs in tqdm(chunk_list(df.index.tolist(), BATCH_SIZE),
                       desc="Scoring batches", unit="batch"):
    batch_df = df.loc[batch_idxs]
    batch_results, usage = evaluate_batch(batch_df)
    results.extend(batch_results)

    # Estimate cost
    in_tokens  = usage.prompt_tokens
    out_tokens = usage.completion_tokens
    total_cost += (in_tokens  / 1000 * PRICE_PER_1K_INPUT) \
                + (out_tokens / 1000 * PRICE_PER_1K_OUTPUT)

Scoring batches:   0%|          | 0/27 [00:00<?, ?batch/s]

In [19]:
# 7. Merge, Score, Sort, and Export
scores_df = pd.DataFrame(results)
combined = df.merge(scores_df, on="BoothNumber")
score_cols = ["Originality", "ScientificRigor", "Clarity", "Relevance", "Feasibility"]
combined["TotalScore"] = combined[score_cols].sum(axis=1)
combined.sort_values(by="TotalScore", ascending=False, inplace=True)

combined.to_csv(output_path, index=False)

In [20]:
# 8. Summary
elapsed = time.time() - start_time
print(f"✅ Completed in {elapsed:.1f}s; Approx. API cost: ${total_cost:.3f}")
combined.head(10)

✅ Completed in 1807.1s; Approx. API cost: $0.775


,BoothNumber,ParentCategory,ProjectTitle,AbstractText,Originality,ScientificRigor,Clarity,Relevance,Feasibility,TotalScore
200,BMED038,Biomedical and Health Sciences,Targeting TP53 Methylation Inhibition With RG1...,Cancer has remained the leading cause of death...,9,9,9,10,9,46
1185,SOFT038,Systems Software,Computer Vision Driven Kinematic Analysis of A...,Sabre fencing is a sport of split-second decis...,9,9,9,9,9,45
1313,TMED085,Translational Medical Science,An End-to-End AI Hardware Solution for Ophthal...,The prevalence of diseases such as diabetic re...,9,9,9,9,8,44
239,BMED077,Biomedical and Health Sciences,Investigating the Influence of Coal Ash Heavy ...,As global energy needs have skyrocketed in the...,9,9,9,9,8,44
242,BMED080,Biomedical and Health Sciences,The Effect of Sleep Deprivation on the Brains...,Sleep is a critical physiological process nece...,9,9,9,9,8,44
1201,SOFT054,Systems Software,A Novel Union-Find-Based Decoding Algorithm fo...,Fault tolerant quantum computing (FTQC) provid...,9,9,9,9,8,44
1199,SOFT052T,Systems Software,CO-Sign Language,Individuals with conditions such as Amyotrophi...,9,9,9,9,8,44
241,BMED079T,Biomedical and Health Sciences,Tonsillar Mesenchymal Stem Cells Derived Cream...,Duchenne Muscular Dystrophy (DMD) is a severe ...,9,9,9,9,8,44
1198,SOFT051,Systems Software,Enhancing Diagnostic Reliability and Safety in...,"Diabetic retinopathy (DR), a leading cause of ...",9,9,9,9,8,44
240,BMED078T,Biomedical and Health Sciences,Kera Treatment: Treating Advanced Cases of Ker...,Late-stage keratoconus is a progressive cornea...,9,9,9,9,8,44
